In [ ]:
import torch
import numpy as np

from utils import load_dataset, evaluate, compute_AUCs

In [ ]:
PATH = 'Trained Models/naiveconv.pt'

model = torch.load(PATH, map_location=torch.device('cuda'))
model.eval()

In [ ]:
_, _, test_loader = load_dataset(64)

In [ ]:
pos_weight = torch.tensor([9.719982661465107,
                                 40.447330447330444,
                                 8.425640640264522,
                                 5.6423934376729905,
                                 19.512704490080054,
                                 17.73208919816543,
                                 82.86770140428678,
                                 21.162702906757268,
                                 24.023998285836726,
                                 48.68432479374729,
                                 44.56279809220986,
                                 66.5005931198102,
                                 33.122599704579024,
                                 493.92070484581495])

criterion =  torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
device = 'cuda'

eval_dict  = evaluate(model, test_loader, criterion, device)

In [ ]:
N_CLASSES = 14
CLASS_NAMES = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
               'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']

print(f"Eval loss is:{eval_dict['loss']}")
print(f"Eval accuracy is:{eval_dict['acc']}")
print(f"Eval f1 score is:{eval_dict['f1']}")

AUROCs = compute_AUCs(eval_dict['labels'], eval_dict['logits'])
AUROC_avg = np.array(AUROCs).mean()
print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
for i in range(N_CLASSES):
    print('The AUROC of {} is {}'.format(CLASS_NAMES[i], AUROCs[i]))